In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score


In [ ]:


data= tf.keras.datasets.cifar10
(train_image,train_label),(test_image,test_label)= data.load_data()
print(train_image.shape)
print(test_image.shape)
train_labels = to_categorical(train_label)
test_labels = to_categorical(test_label)
train_image= train_image/255.0
test_image= test_image/255.0
 


In [ ]:
vgg16_model= VGG16(weights='imagenet',
                    include_top=False, 
                    classes=10,
                    input_shape=(32,32,3))

model = tf.keras.models.Sequential()

for i in range(len(vgg16_model.layers)):
    model.add(vgg16_model.layers[i])

for i in range(len(model.layers)): 
    model.layers[i].trainable = False

from tensorflow.keras.layers import Dense, Flatten, Dropout
model.add(Flatten())
model.add(Dense(512, activation='relu', name='hidden1'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu', name='hidden2'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax', name='predictions'))

model.summary()

In [ ]:

model.compile(loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'], optimizer= 'adam')
ES = EarlyStopping(patience=5, monitor='val_accuracy', mode='max')



In [ ]:


AUG = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                        horizontal_flip=True, fill_mode="nearest")
 
history = model.fit_generator(AUG.flow(train_image,train_label, batch_size=BS),validation_data=(test_image,test_label), epochs =100,
                             callbacks=[ES])


In [ ]:
predictions = model.predict(test_image)

train_loss, train_score = model.evaluate(train_image, train_label)
test_loss, test_score = model.evaluate(test_image, test_label)
print("Train Loss:", train_loss)
print("Test Loss:", test_loss)
print("Train Score:", train_score)
print("Test Score:", test_score)
